# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [20]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [21]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [22]:
top_250 = imdb.top_250()

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [23]:
top_250 = pd.DataFrame(top_250)
top_250.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1660365,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1136449,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,776176,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1646039,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,849441,8.9,tt0108052,Schindler's List,feature,1993


In [24]:
del top_250['type']
del top_250['can_rate']
del top_250['image']

#### 3. Select only the top 100 movies

In [25]:
top_100 = top_250[:100]
top_100

,num_votes,rating,tconst,title,year
0,1660365,9.3,tt0111161,The Shawshank Redemption,1994
1,1136449,9.2,tt0068646,The Godfather,1972
2,776176,9.0,tt0071562,The Godfather: Part II,1974
3,1646039,9.0,tt0468569,The Dark Knight,2008
4,849441,8.9,tt0108052,Schindler's List,1993
5,437963,8.9,tt0050083,12 Angry Men,1957
6,1301277,8.9,tt0110912,Pulp Fiction,1994
7,1194859,8.9,tt0167260,The Lord of the Rings: The Return of the King,2003
8,494950,8.9,tt0060196,"The Good, the Bad and the Ugly",1966
9,1321965,8.9,tt0137523,Fight Club,1999


#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [26]:
runtime = []
for x in top_100['tconst']:
    title = imdb.get_title_by_id(x)
    runtimes = title.runtime
    runtime.append(runtimes)
runtime[0:5]

[8520, 10500, 12120, 9120, 11700]

In [27]:
top_100['runtime'] = runtime

/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
top_100.head()

,num_votes,rating,tconst,title,year,runtime
0,1660365,9.3,tt0111161,The Shawshank Redemption,1994,8520
1,1136449,9.2,tt0068646,The Godfather,1972,10500
2,776176,9.0,tt0071562,The Godfather: Part II,1974,12120
3,1646039,9.0,tt0468569,The Dark Knight,2008,9120
4,849441,8.9,tt0108052,Schindler's List,1993,11700


In [31]:
genre = []
for x in top_100['tconst']:
    title = imdb.get_title_by_id(x)
    genres = title.genres
    genre.append(genres)
genre[0:5]

[[u'Crime', u'Drama'],
 [u'Crime', u'Drama'],
 [u'Crime', u'Drama'],
 [u'Action', u'Crime', u'Thriller'],
 [u'Biography', u'Drama', u'History']]

In [ ]:
top_100['genre'] = genre

In [ ]:
top_100.head()

#### 4. Write the Results to a csv

In [15]:
top_100.to_csv("../assets/data.top_100.csv", encoding="utf8")

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [42]:
import requests
from scrapy.selector import Selector

In [ ]:
#This is the code to scrape just the review on the main page of a film
# main_reviews = []
# for x in top_100['tconst']:
#     response = requests.get('http://www.imdb.com/title/%s/'% x)
#     HTML = response.text
#     review = Selector(text=HTML).xpath('//*[@id="titleUserReviewsTeaser"]/div/span/div[2]/p').extract()
#     main_reviews.append(review)

In [ ]:
#main_reviews[96]

In [43]:
#This is code to scrape the first page of the reviews section for a film.
first_page_reviews = []
for x in top_100['tconst']:
    response = requests.get('http://www.imdb.com/title/%s/reviews?ref_=tt_urv' % x)
    HTML = response.text
    review_first_page = Selector(text=HTML).xpath('//*[@id="tn15content"]/p').extract()
    first_page_reviews.append(review_first_page)

In [ ]:
# first_page_reviews[96]

In [ ]:
# # Code to scrape the star ratings on user reivews
# ratings = []
# for x in top_100['tconst']:
#     response = requests.get('http://www.imdb.com/title/%s/reviews?ref_=tt_urv' % x)
#     HTML = response.text
#     rating = Selector(text=HTML).xpath('//*[@id="tn15content"]/div/img/@alt').extract()
#     ratings.append(rating)

In [ ]:
#ratings[34]

#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

In [33]:
user_review_total = []
for x in top_100['tconst']:
    user_review = imdb.get_title_reviews(x, max_results=100)
    user_review_total.append(user_review)
    
   
    

In [37]:
reviews = []
ratings = []
for x in top_100['tconst']:
    user_review = imdb.get_title_reviews(x, max_results=100)
    for items in user_review:
        a = repr(items.text)
        reviews.append(a)
        b = repr(items.rating)
        ratings.append(b)

In [46]:
reviews[34]

'u"It\'s times like these that almost tempt me to grab Jigsaw off the shelf. It\'s been almost a month since I saw a film that was flatout bad; the worst review I\'ve given in weeks was a 6/10 to Flightplan. This film obviously didn\'t break my streak.\\n\\nI went into it knowing next to nothing, only that IMDb has it ranked as the second best movie of all time, that it was based on a story by Stephen King, and that it took place in a prison. Needless to say, it blew me away. I wont say a thing about the plot, just so that others who\'ve yet to see this film have a chance at the same experience as me. I will say this though: It has some of the most subtle and unexpected twists you will ever see. One in particular stands out above the rest.\\n\\nNot much can be said about the actors that hasn\'t been said. Morgan Freeman gives one of the best performances of his career, and even though I\'ve only seen 2 or 3 other Tim Robbins movies, I\'m sure the same can be said for him.\\n\\nThough I

In [53]:
import re
clean_review = []
for x in reviews:
    review = re.sub('[^0-9a-zA-Z]+', ' ', x)
    clean_review.append(review)

In [54]:
clean_review[34]

'u It s times like these that almost tempt me to grab Jigsaw off the shelf It s been almost a month since I saw a film that was flatout bad the worst review I ve given in weeks was a 6 10 to Flightplan This film obviously didn t break my streak n nI went into it knowing next to nothing only that IMDb has it ranked as the second best movie of all time that it was based on a story by Stephen King and that it took place in a prison Needless to say it blew me away I wont say a thing about the plot just so that others who ve yet to see this film have a chance at the same experience as me I will say this though It has some of the most subtle and unexpected twists you will ever see One in particular stands out above the rest n nNot much can be said about the actors that hasn t been said Morgan Freeman gives one of the best performances of his career and even though I ve only seen 2 or 3 other Tim Robbins movies I m sure the same can be said for him n nThough I don t agree with the multitudes 

#### 3. Remove the non AlphaNumeric characters from reviews

In [55]:
import re
clean_review = []
for x in reviews:
    review = re.sub('[^0-9a-zA-Z]+', ' ', x)
    clean_review.append(review)

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### 5. Merge the user reviews and ratings

# I had a difficult time figuring out the scraping and getting the API to work to pull the reviews so I did not get very far in the project yet.

#### 6. Save this merged dataframe as a csv

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_